In [1]:
!pip install --upgrade pip
!pip install transformers==4.40.1 datasets==2.19.0 accelerate==0.30.0 peft==0.10.0 bitsandbytes==0.43.1 -qqq
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 47.3

In [2]:
import transformers
import datasets
import accelerate
import peft
import bitsandbytes
import warnings

warnings.filterwarnings('ignore')

# **메모리 사용량 측정을 위한 함수 구현**

In [3]:
import torch

def print_gpu_utilization():
  # torch.cuda.is_available() -> GPU 사용 여부
  if torch.cuda.is_available():
    # torch.cuda.memory_allocated() -> 사용 메모리
    used_memory = torch.cuda.memory_allocated() / 1024 ** 3
    print(f"GPU 메모리 사용량: {used_memory: .3f} GB")
  else:
    print("런타임 유형을 GPU로 변경하세요")

print_gpu_utilization()
# 출력 결과
# GPU 메모리 사용량: 0.000 GB

런타임 유형을 GPU로 변경하세요


# **모델을 불러오고 GPU 메모리와 데이터 타입 확인**

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model_and_tokenizer(model_id, peft=None):
  tokenizer = AutoTokenizer.from_pretrained(model_id)

  if peft is None:
    # AutoModelForCausalLM -> 예측형 언어 모델 (GPT 등)
    # torch_dtype="auto" -> PyTorch에서 자동으로 적절한 데이터 타입(float16 또는 float32)을 선택하도록 설정
    # device_map={"": 0} -> 모델을 GPU 0번 장치에 로드 -> auto로 설정하면 여러 GPU에 자동 배치
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

  print_gpu_utilization()

  return model, tokenizer

model_id = "EleutherAI/polyglot-ko-1.3b"
model, tokenizer = load_model_and_tokenizer(model_id) # GPU 메모리 사용량: 2.599 GB
print("모델 파라미터 데이터 타입: ", model.dtype) # torch.float16

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/748M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: Cannot access accelerator device when none is available.

# **그레이디언트와 옵티마이저 상태의 메모리 사용량을 계산하는 함수**

In [5]:
from transformers import AdamW
from torch.utils.data import DataLoader

def estimate_memory_of_gradients(model):
  total_memory = 0

  # model.parameters() -> 모델의 모든 가중치(파라미터)를 가져옴
  # param -> 모델의 각 가중치 텐서
  for param in model.parameters():
    # PyTorch에서는 역전파 후에만 param.grad가 존재
    # 만약 param.grad이 None 이면 해당 파라미터에 대한 그래디언트가 아직 계산되지 않은 것
    if param.grad is not None:
      # param.grad.nelement() -> 텐서의 요소 개수 (총 몇 개의 숫자가 있는지)
      # param.grad.element_size() -> 텐서 한 개 요소가 차지하는 바이트 수
      total_memory += param.grad.nelement() * param.grad.element_size()

  return total_memory

def estimate_memory_of_optimizer(optimizer):
  total_memory = 0

  # optimizer.state -> 옵티마이저가 각 파라미터에 대해 저장하는 추가 정보가 포함(모멘텀 등)
  # .values() -> 값 가져오기
  for state in optimizer.state.values():
    # state -> 딕셔너리 형태
    for k, v in state.items():
      # 옵티마이저가 저장하는 값 중에서 텐서인 경우만 메모리 계산
      if torch.is_tensor(v):
        total_memory += v.nelement() * v.element_size()

  return total_memory

# **모델의 학습 과정에서 메모리 사용량을 확인하는 train_model 정의**

In [ ]:
def train_model(model, dataset, training_args):
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    model.train()
    gpu_utilization_printed = False

    for step, batch in enumerate(train_dataloader, start=1):
        # 배치 아이템을 불러와 값을 GPU로 이동
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # 모델 학습
        outputs = model(**batch)
        loss = outputs.loss
        # Gradient Accumulation을 사용하면 한 번의 업데이트를 위해 여러 개의 배치에서 손실을 누적
        # 하지만 이렇게 하면 손실이 여러 배 누적되므로 역전파 전에 손실을 나눈다

        # gradient_accumulation_steps -> 몇 번의 배치마다 한 번씩 옵티마이저를 업데이트할지 결정하는 값
        loss = loss / training_args.gradient_accumulation_steps
        loss.backward()

        # gradient_accumulation_steps 번 만큼 그래디언트를 누적한 후 가중치를 업데이트하도록 설정
        # gradient_accumulation_steps = 4이면 4번의 배치마다 한 번씩 optimizer.step() 실행
        if step % training_args.gradient_accumulation_steps == 0:
            # 가중치 업데이트
            optimizer.step()
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)

            # GPU 사용량을 처음 한 번만 출력한다
            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True

            # 옵티마이저의 그래디언트 초기화
            # loss.backword()가 실행될 때마다 그래디언트가 누적되므로 옵티마이저를 업데이트한 후에는
            # 반드시 zero_grad()를 호출해야 한다 그렇지 않으면 이전 배치의 그래디언트가 남아 있어 학습이 제대로 이루어지지 않는다
            optimizer.zero_grad()

    print(f"옵티마이저 상태의 메모리 사용량: {optimizer_memory / (1024 ** 3):.3f} GB")
    print(f"그레디언트 메모리 사용량: {gradients_memory / (1024 ** 3):.3f} GB")

# **랜덤 데이터셋을 생성하는 make_dummy_dataset 정의**

In [ ]:
import numpy as np
from datasets import Dataset

def make_dummy_dataset():
  seq_len, dataset_size = 256, 64

  dummy_data = {
      # 2차원 배열 생성
      # (생성할 랜덤 정수의 최소값, 최대값 (행, 열))
      "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
      "labels": np.random.randint(100, 30000, (dataset_size, seq_len)),
  }

  # 딕셔너리를 데이터셋으로 만들고 텐서로 변환
  dataset = Dataset.from_dict(dummy_data)
  dataset.set_format("pt")

  return dataset

# **더이상 사용하지 않는 GPU 메모리를 반환하는 cleanup 함수**

In [6]:
# 파이썬 가비지 컬렉터
import gc

def cleanup():
  # 글로벌 변수(전역 변수)들을 딕셔너리로 관리한다
  if 'model' in globals():
    # 변수를 삭제하여 메모리 해제
    del globals()['model']
  if 'dataset' in globals():
    del globals()['dataset']

  # 더 이상 사용하지 않는 객체(변수)를 강제로 정리
  # 메모리에서 삭제된 객체들이 즉시 해제되도록 유도
  gc.collect()
  # GPU 메모리 캐시 비우기
  # PyTorch는 메모리를 즉시 해제하지 않고 캐시에 보관하는 특성이 있다
  # GPU VRAM을 확보하려면 사용하지 않는 메모리를 비워줘야 함
  torch.cuda.empty_cache()

# **GPU 사용량을 확인하는 gpu_memory_experiment 함수 정의**

In [7]:
from transformers import TrainingArguments, Trainer

def gpu_memory_experiment(batch_size,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    model_id="EleutherAI/polyglot-ko-1.3b",
    peft=None):

  print(f"배치 사이즈: {batch_size}")
  model, tokenizer = load_model_and_tokenizer(model_id, peft=peft)

  if gradient_checkpointing == True or peft == 'qlora':
    model.config.use_cache = False

  dataset = make_dummy_dataset()

  training_args = TrainingArguments(
      per_device_train_batch_size = batch_size,
      gradient_accumulation_steps = gradient_accumulation_steps,
      gradient_checkpointing = gradient_checkpointing,
      output_dir="./result",
      num_train_epochs = 1
  )

  try:
    train_model(model, dataset, training_args)
  except RuntimeError as e:
    if "CUDA out of memory" in str(e):
      print(e)
    else:
      raise e
  finally:
    # 작업이 끝난 뒤 삭제 + 메모리 회수 후 출력
    del model, dataset
    gc.collect()
    torch.cuda.empty_cache()
    print_gpu_utilization()



# **배치 사이즈를 변경하며 메모리 사용량 측정**

In [ ]:
cleanup()
print_gpu_utilization()

for batch_size in [4, 8, 16]:
  gpu_memory_experiment(batch_size)

  torch.cuda.empty_cache()

# **train_model에서 그레이디언트 누적과 관련된 부분 설명**

In [8]:
def train_model(model, dataset, training_args):
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    model.train()
    gpu_utilization_printed = False

    for step, batch in enumerate(train_dataloader, start=1):
        # 배치 아이템을 불러와 값을 GPU로 이동
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # 모델 학습
        outputs = model(**batch)
        loss = outputs.loss
        # Gradient Accumulation을 사용하면 한 번의 업데이트를 위해 여러 개의 배치에서 손실을 누적
        # 하지만 이렇게 하면 손실이 여러 배 누적되므로 역전파 전에 손실을 나눈다

        # gradient_accumulation_steps -> 몇 번의 배치마다 한 번씩 옵티마이저를 업데이트할지 결정하는 값
        loss = loss / training_args.gradient_accumulation_steps
        loss.backward()

        # gradient_accumulation_steps 번 만큼 그래디언트를 누적한 후 가중치를 업데이트하도록 설정
        # gradient_accumulation_steps = 4이면 4번의 배치마다 한 번씩 optimizer.step() 실행
        if step % training_args.gradient_accumulation_steps == 0:
            # 가중치 업데이트
            optimizer.step()
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)

            # GPU 사용량을 처음 한 번만 출력한다
            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True

            # 옵티마이저의 그래디언트 초기화
            # loss.backword()가 실행될 때마다 그래디언트가 누적되므로 옵티마이저를 업데이트한 후에는
            # 반드시 zero_grad()를 호출해야 한다 그렇지 않으면 이전 배치의 그래디언트가 남아 있어 학습이 제대로 이루어지지 않는다
            optimizer.zero_grad()

# **그레이디언트 누적을 적용했을 때 메모리 사용량**

In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=4, gradient_accumulation_steps=4)

torch.cuda.empty_cache()
# 출력 결과
# GPU 메모리 사용량: 2.615 GB
# GPU 메모리 사용량: 10.586 GB
# 옵티마이저 상태의 메모리 사용량: 4.961 GB
# 그레이디언트 메모리 사용량: 2.481 GB

# **그레이디언트 체크포인팅 사용 시 메모리 사용량**

In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, gradient_checkpointing=True)

torch.cuda.empty_cache()
# 출력 결과
# GPU 메모리 사용량: 10.290 GB
# 옵티마이저 상태의 메모리 사용량: 4.961 GB
# 그레이디언트 메모리 사용량: 2.481 GB

# **모델을 불러오면서 LoRA 적용하기**

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

def load_model_and_tokenizer(model_id, peft=None):
  tokenizer = AutoTokenizer.from_pretrained(model_id)

  if peft is None:
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
  elif peft == 'lora':
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    # LoRA 설정을 정의
    lora_config = LoraConfig(
        # LoRA의 rank 값 (저차원 행렬 크기, 작을수록 메모리 절약)
        r=8,
        # LoRA의 학습률 조정 계수 (클수록 학습 영향 증가)
        lora_alpha=32,
        # LoRA를 적용할 레이어 -> "query_key_value" -> 트랜스포머 내부의 QKV 어텐션 부분
        target_modules=["query_key_value"],
        lora_dropout=0.05,
        # 추가적인 바이어스 파라미터를 학습하지 않음
        bias="none",
        # 이 모델이 Causal Language Model (GPT 계열) 임을 명시
        task_type="CAUSAL_LM"
    )

    # get_peft_model(model, lora_config) -> 기존 모델에 LoRA를 적용하여 경량화된 파인튜닝 모델을 생성
    # 기존의 모든 가중치를 업데이트하는 대신 LoRA가 추가된 작은 행렬만 학습하도록 변환
    model = get_peft_model(model, lora_config)

    # print_trainable_parameters() -> 현재 학습 가능한(업데이트 할 수 있는) 파라미터 수를 출력
    model.print_trainable_parameters()

  print_gpu_utilization()
  return model, tokenizer

# **LoRA를 적용했을 때 GPU 메모리 사용량 확인**

In [1]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft="lora")

torch.cuda.empty_cache()

# GPU 메모리 사용량: 0.016 GB
# 배치 사이즈: 16
# Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
# Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]
# trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.11796039111242178
# GPU 메모리 사용량: 2.618 GB
# GPU 메모리 사용량: 4.732 GB
# 옵티마이저 상태의 메모리 사용량: 0.006 GB
# 그레디언트 메모리 사용량: 0.003 GB
# GPU 메모리 사용량: 0.016 GB

NameError: name 'cleanup' is not defined